In [1]:
import pandas as pd
import numpy as np

import ast
import json
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
df = pd.read_csv('../data/movies_metadata.csv')

/var/folders/wk/xh1t9b793w52fjy9qr_0q3pr0000gn/T/ipykernel_12594/2089594497.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


In [3]:
df = df[['overview', 'genres']]

In [4]:
# Clean up the genres column
df['genres'] = df['genres'].fillna('[]') \
                          .apply(ast.literal_eval) \
                          .apply(lambda x: [i['name'] for i in x])

df

,overview,genres
0,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]"
1,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]"
2,A family wedding reignites the ancient feud be...,"[Romance, Comedy]"
3,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]"
4,Just when George Banks has recovered from his ...,[Comedy]
...,...,...
45461,Rising and falling between a man and woman.,"[Drama, Family]"
45462,An artist struggles to finish his work while a...,[Drama]
45463,"When one of her hits goes wrong, a professiona...","[Action, Drama, Thriller]"
45464,"In a small town live two brothers, one a minis...",[]


In [6]:
# Convert genres into binary format
mlb = MultiLabelBinarizer()
df_genres = pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                          columns=mlb.classes_,
                          index=df.index)


In [7]:
df_genres

,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,Drama,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
45462,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
45463,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
45464,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Filter out genres with less than 10 occurrences
genre_counts = df_genres.sum()
popular_genres = genre_counts[genre_counts >= 10].index
df_genres = df_genres[popular_genres]


In [10]:
df = df.join(df_genres)
df

,overview,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,"Led by Woody, Andy's toys live happily in his ...",0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,When siblings Judy and Peter discover an encha...,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,A family wedding reignites the ancient feud be...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,"Cheated on, mistreated and stepped on, the wom...",0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Just when George Banks has recovered from his ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,Rising and falling between a man and woman.,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
45462,An artist struggles to finish his work while a...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
45463,"When one of her hits goes wrong, a professiona...",1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
45464,"In a small town live two brothers, one a minis...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.columns

Index(['overview', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [13]:
len(popular_genres)

20